In [2]:
import os
import json
import time
import base64
import pathlib
from typing import Dict, List, Optional, Tuple

import requests
import numpy as np
import pandas as pd
import tensorflow as tf

# Config for BC

In [ ]:
OWNER = "Triss11"
REPO  = "FL"
REF   = "main"

# Download EXACTLY these two files:
FNN_MODEL_PATH  = "M.Tech_Dissertation/Server/global_model/global_FNN_model.keras"
LSTM_MODEL_PATH = "M.Tech_Dissertation/Server/global_model/global_lstm_model.keras"

# User data directories
FNN_USERS_DIR  = pathlib.Path("/Users/sohinikar/FL/M.Tech_Dissertation/Client/data/FNN_BC_test_data")
LSTM_USERS_DIR = pathlib.Path("/Users/sohinikar/FL/M.Tech_Dissertation/Client/data/LSTM_BC_test_data")

# Output roots (per-user subfolders created automatically)
FNN_OUT_ROOT  = pathlib.Path("/Users/sohinikar/FL/M.Tech_Dissertation/Client/client_params/FNN_BC")
LSTM_OUT_ROOT = pathlib.Path("/Users/sohinikar/FL/M.Tech_Dissertation/Client/client_params/LSTM_BC")

# Training hyperparams
EPOCHS = 1
BATCH_SIZE = 8
OPTIMIZER = "adam"
METRICS = ["accuracy"]

In [ ]:
def gh_session() -> requests.Session:
    s = requests.Session()
    token = os.environ.get("GITHUB_TOKEN")
    if token:
        s.headers.update({"Authorization": f"Bearer {token}"})
    s.headers.update({"Accept": "application/vnd.github+json"})
    return s


def get_github_file_entry(session: requests.Session, owner: str, repo: str, path: str, ref: str = "main") -> Dict:
    """Return the GitHub contents API JSON for a specific file path."""
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{path}?ref={ref}"
    r = session.get(url)
    if r.status_code != 200:
        raise RuntimeError(f"Could not fetch {path}: {r.status_code} {r.text}")
    entry = r.json()
    if entry.get("type") != "file" or not entry.get("download_url"):
        raise RuntimeError(f"{path} is not a downloadable file.")
    return entry


def download_file(session: requests.Session, entry: Dict, dest: pathlib.Path) -> pathlib.Path:
    """Download a file using 'download_url' from the GitHub contents API."""
    url = entry.get("download_url")
    if not url:
        raise RuntimeError(f"No download_url for {entry.get('name')}")
    dest.parent.mkdir(parents=True, exist_ok=True)
    r = session.get(url, stream=True)
    if r.status_code != 200:
        raise RuntimeError(f"Download failed for {entry.get('name')}: {r.status_code} {r.text}")
    with open(dest, "wb") as f:
        for chunk in r.iter_content(chunk_size=1024 * 1024):
            if chunk:
                f.write(chunk)
    return dest


def load_user_dirs(root: pathlib.Path) -> List[pathlib.Path]:
    """Return sorted list of user_* directories that contain X.csv and y.csv."""
    if not root.exists():
        return []
    users = []
    for p in sorted(root.glob("user_*")):
        if (p / "X.csv").exists() and (p / "y.csv").exists():
            users.append(p)
    return users


def _auto_pick_loss_and_cast_labels(model: tf.keras.Model, y: np.ndarray) -> (str, np.ndarray):
    """
    Choose an appropriate loss based on the model output shape.
    - If single output -> binary_crossentropy (float labels 0/1)
    - If multi-class outputs -> sparse_categorical_crossentropy (int labels 0..C-1)
    """
    out_shape = model.output_shape
    if isinstance(out_shape, (list, tuple)):
        out_shape = out_shape[0]
    if isinstance(out_shape, (list, tuple)) and len(out_shape) > 1:
        num_outputs = int(np.prod(out_shape[1:]))
    else:
        num_outputs = 1

    if num_outputs == 1:
        y = y.astype(np.float32).reshape(-1)
        return "binary_crossentropy", y
    else:
        y = y.astype(np.int32).reshape(-1)
        return "sparse_categorical_crossentropy", y


def train_one_user(model_path: pathlib.Path,
                   user_dir: pathlib.Path,
                   out_dir: pathlib.Path,
                   is_lstm: bool) -> Dict:
    """
    Load model, train on user data (no validation), save weights + params.json to out_dir.
    Automatically chooses the correct loss based on model output shape.
    """
    # Load data
    X = pd.read_csv(user_dir / "X.csv").to_numpy(dtype=np.float32)
    y = pd.read_csv(user_dir / "y.csv")["label"].to_numpy()

    if is_lstm:
        X = X.reshape((X.shape[0], 1, X.shape[1])).astype(np.float32)

    # Load model and pick loss
    model = tf.keras.models.load_model(str(model_path))
    chosen_loss, y = _auto_pick_loss_and_cast_labels(model, y)

    # Compile fresh (weights may not include optimizer state)
    model.compile(optimizer=OPTIMIZER, loss=chosen_loss, metrics=METRICS)

    # Train (no validation)
    start = time.time()
    hist = model.fit(X, y, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)
    elapsed = time.time() - start

    # Prepare output
    out_dir.mkdir(parents=True, exist_ok=True)
    weights_path = out_dir / f"{user_dir.name}.weights.h5"   # Keras 3 requires .weights.h5
    model.save_weights(str(weights_path))

    final_metrics = {f"final_{k}": float(v[-1]) for k, v in hist.history.items()}
    params = {
        "user": user_dir.name,
        "model_file": model_path.name,
        "samples": int(X.shape[0]),
        "features": int(X.shape[-1]),
        "is_lstm": is_lstm,
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "optimizer": OPTIMIZER,
        "loss": chosen_loss,
        "metrics": METRICS,
        "train_seconds": round(elapsed, 3),
        **final_metrics
    }
    with open(out_dir / "params.json", "w") as f:
        json.dump(params, f, indent=2)

    return {"weights": str(weights_path), "params": params}


def main():
    session = gh_session()

    # 1) Fetch exact global models from GitHub
    print("🔎 Fetching exact global models from GitHub…")
    fnn_entry  = get_github_file_entry(session, OWNER, REPO, FNN_MODEL_PATH, REF)
    lstm_entry = get_github_file_entry(session, OWNER, REPO, LSTM_MODEL_PATH, REF)

    # 2) Download them locally (cache under ./_downloaded_models)
    cache_dir = pathlib.Path("./_downloaded_models")
    fnn_local  = download_file(session, fnn_entry,  cache_dir / pathlib.Path(FNN_MODEL_PATH).name)
    lstm_local = download_file(session, lstm_entry, cache_dir / pathlib.Path(LSTM_MODEL_PATH).name)
    print(f"⬇️  Downloaded FNN model:  {fnn_local}")
    print(f"⬇️  Downloaded LSTM model: {lstm_local}")

    # 3) Enumerate users
    fnn_users  = load_user_dirs(FNN_USERS_DIR)
    lstm_users = load_user_dirs(LSTM_USERS_DIR)

    if not fnn_users:
        print(f"⚠️  No valid FNN user folders found under: {FNN_USERS_DIR}")
    if not lstm_users:
        print(f"⚠️  No valid LSTM user folders found under: {LSTM_USERS_DIR}")

    # Collect times for summary
    fnn_times: List[float] = []
    lstm_times: List[float] = []

    # 4) Train FNN users
    print(f"🏃 Training FNN users ({len(fnn_users)}) …")
    for udir in fnn_users:
        out_dir = FNN_OUT_ROOT / udir.name
        try:
            result = train_one_user(fnn_local, udir, out_dir, is_lstm=False)
            secs = float(result["params"]["train_seconds"])
            fnn_times.append(secs)
            print(f"✅ {udir.name}: saved -> {result['weights']}  ({secs:.3f}s)")
        except Exception as e:
            print(f"❌ {udir.name}: {e}")

    # 5) Train LSTM users
    print(f"🏃 Training LSTM users ({len(lstm_users)}) …")
    for udir in lstm_users:
        out_dir = LSTM_OUT_ROOT / udir.name
        try:
            result = train_one_user(lstm_local, udir, out_dir, is_lstm=True)
            secs = float(result["params"]["train_seconds"])
            lstm_times.append(secs)
            print(f"✅ {udir.name}: saved -> {result['weights']}  ({secs:.3f}s)")
        except Exception as e:
            print(f"❌ {udir.name}: {e}")

    # 6) Print summaries
    if fnn_times:
        print("\n⏱️ FNN training time summary:")
        print(f"   Users trained: {len(fnn_times)}")
        print(f"   Avg: {sum(fnn_times)/len(fnn_times):.3f}s   Min: {min(fnn_times):.3f}s   Max: {max(fnn_times):.3f}s")

    if lstm_times:
        print("\n⏱️ LSTM training time summary:")
        print(f"   Users trained: {len(lstm_times)}")
        print(f"   Avg: {sum(lstm_times)/len(lstm_times):.3f}s   Min: {min(lstm_times):.3f}s   Max: {max(lstm_times):.3f}s")

    print("\n🎉 Done.")


if __name__ == "__main__":
    main()

🔎 Fetching exact global models from GitHub…
⬇️  Downloaded FNN model:  _downloaded_models/global_FNN_model.keras
⬇️  Downloaded LSTM model: _downloaded_models/global_lstm_model.keras
🏃 Training FNN users (128) …
✅ user_001: saved -> /Users/sohinikar/FL/M.Tech_Dissertation/Client/client_params/FNN_BC/user_001/user_001.weights.h5  (0.457s)
✅ user_002: saved -> /Users/sohinikar/FL/M.Tech_Dissertation/Client/client_params/FNN_BC/user_002/user_002.weights.h5  (0.413s)
✅ user_003: saved -> /Users/sohinikar/FL/M.Tech_Dissertation/Client/client_params/FNN_BC/user_003/user_003.weights.h5  (0.420s)
✅ user_004: saved -> /Users/sohinikar/FL/M.Tech_Dissertation/Client/client_params/FNN_BC/user_004/user_004.weights.h5  (0.383s)
✅ user_005: saved -> /Users/sohinikar/FL/M.Tech_Dissertation/Client/client_params/FNN_BC/user_005/user_005.weights.h5  (0.374s)
✅ user_006: saved -> /Users/sohinikar/FL/M.Tech_Dissertation/Client/client_params/FNN_BC/user_006/user_006.weights.h5  (0.370s)
✅ user_007: saved ->

In [26]:
#!/usr/bin/env python3
"""
Client-side trainer for per-user updates (Multiclass: FNN + LSTM) with robust loss/metric handling.

Downloads EXACT models from:
  - M.Tech_Dissertation/Server/global_model/global_FNN_MC_model.keras
  - M.Tech_Dissertation/Server/global_model/global_lstm_MC_model.keras

Trains on users found under:
  - /Users/sohinikar/FL/M.Tech_Dissertation/Client/data/FNN_MC_test_data/user_*/
  - /Users/sohinikar/FL/M.Tech_Dissertation/Client/data/LSTM_MC_test_data/user_*/

Saves per-user outputs to:
  - /Users/sohinikar/FL/M.Tech_Dissertation/Client/client_params/FNN_MC/user_xxx/
  - /Users/sohinikar/FL/M.Tech_Dissertation/Client/client_params/LSTM_MC/user_xxx/

Also prints average/min/max train times per family.

Env (if repo is private): export GITHUB_TOKEN=...
Deps: pip install tensorflow pandas requests
"""

import os, json, time, pathlib
from typing import Dict, List, Tuple, Optional

import numpy as np
import pandas as pd
import requests
import tensorflow as tf

# ---------------- CONFIG (Multiclass) ----------------
OWNER = "Triss11"
REPO  = "FL"
REF   = "main"

FNN_MODEL_PATH  = "M.Tech_Dissertation/Server/global_model/global_FNN_MC_model.keras"
LSTM_MODEL_PATH = "M.Tech_Dissertation/Server/global_model/global_lstm_MC_model.keras"

FNN_USERS_DIR  = pathlib.Path("/Users/sohinikar/FL/M.Tech_Dissertation/Client/data/FNN_MC_test_data")
LSTM_USERS_DIR = pathlib.Path("/Users/sohinikar/FL/M.Tech_Dissertation/Client/data/LSTM_MC_test_data")

FNN_OUT_ROOT  = pathlib.Path("/Users/sohinikar/FL/M.Tech_Dissertation/Client/client_params/FNN_MC")
LSTM_OUT_ROOT = pathlib.Path("/Users/sohinikar/FL/M.Tech_Dissertation/Client/client_params/LSTM_MC")

EPOCHS = 5
BATCH_SIZE = 64
OPTIMIZER = "adam"
# -----------------------------------------------------


def gh_session() -> requests.Session:
    s = requests.Session()
    tok = os.environ.get("GITHUB_TOKEN")
    if tok:
        s.headers.update({"Authorization": f"Bearer {tok}"})
    s.headers.update({"Accept": "application/vnd.github+json"})
    return s


def get_github_file_entry(session: requests.Session, owner: str, repo: str, path: str, ref: str = "main") -> Dict:
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{path}?ref={ref}"
    r = session.get(url)
    if r.status_code != 200:
        raise RuntimeError(f"Could not fetch {path}: {r.status_code} {r.text}")
    entry = r.json()
    if entry.get("type") != "file" or not entry.get("download_url"):
        raise RuntimeError(f"{path} is not a downloadable file.")
    return entry


def download_file(session: requests.Session, entry: Dict, dest: pathlib.Path) -> pathlib.Path:
    url = entry.get("download_url")
    dest.parent.mkdir(parents=True, exist_ok=True)
    r = session.get(url, stream=True)
    if r.status_code != 200:
        raise RuntimeError(f"Download failed for {entry.get('name')}: {r.status_code} {r.text}")
    with open(dest, "wb") as f:
        for chunk in r.iter_content(1024 * 1024):
            if chunk:
                f.write(chunk)
    return dest


def load_user_dirs(root: pathlib.Path) -> List[pathlib.Path]:
    if not root.exists():
        return []
    users = []
    for p in sorted(root.glob("user_*")):
        if (p / "X.csv").exists() and (p / "y.csv").exists():
            users.append(p)
    return users


def _infer_output_info(model: tf.keras.Model, X_batch: np.ndarray) -> Tuple[int, bool]:
    """
    Robustly infer (num_outputs, from_logits) by doing a 1-sample forward pass.
    Falls back to last layer .units if needed.
    """
    # Try to build via forward pass
    try:
        yhat = model(tf.convert_to_tensor(X_batch[:1]), training=False)
        y0 = yhat[0] if isinstance(yhat, (list, tuple)) else yhat
        shp = tuple(y0.shape)  # e.g., (1, 4)
        num_outputs = int(np.prod(shp[1:])) if len(shp) > 1 else 1
    except Exception:
        # Fallback: use last layer units if available
        last = model.layers[-1]
        num_outputs = int(getattr(last, "units", 1))

    # Decide logits vs activated from the last layer
    last = model.layers[-1]
    act = getattr(last, "activation", None)
    act_name = getattr(act, "__name__", None)
    from_logits = (act_name in (None, "linear"))

    return num_outputs, from_logits


def _sanitize_labels(y: np.ndarray) -> Tuple[np.ndarray, Optional[Dict[int, int]]]:
    """Ensure y are integer class IDs 0..C-1. Convert strings/floats if needed."""
    y = np.asarray(y)
    if y.dtype.kind in {"U", "S", "O"}:
        classes, inv = np.unique(y, return_inverse=True)
        return inv.astype(np.int32), {i: int(c) for i, c in enumerate(classes)}
    if y.dtype.kind == "f":
        return np.rint(y).astype(np.int32), None
    return y.astype(np.int32), None


def _choose_loss_and_metrics(num_outputs: int, from_logits: bool):
    """Return (loss_fn, metrics_list) objects appropriate for the model."""
    if num_outputs == 1:
        loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=from_logits)
        metrics = [tf.keras.metrics.BinaryAccuracy(name="binary_accuracy")]
    else:
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=from_logits)
        metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name="sparse_categorical_accuracy")]
    return loss_fn, metrics


def train_one_user(model_path: pathlib.Path,
                   user_dir: pathlib.Path,
                   out_dir: pathlib.Path,
                   is_lstm: bool) -> Dict:
    # Load data
    X = pd.read_csv(user_dir / "X.csv").to_numpy(dtype=np.float32)
    y = pd.read_csv(user_dir / "y.csv")["label"].to_numpy()

    if is_lstm:
        X = X.reshape((X.shape[0], 1, X.shape[1])).astype(np.float32)

    # Load model
    model = tf.keras.models.load_model(str(model_path))

    # >>> NEW: infer outputs AFTER we have X shaped, using a forward pass
    num_outputs, from_logits = _infer_output_info(model, X)

    # Sanitize labels
    if num_outputs == 1:
        # Binary
        if y.dtype.kind in {"U", "S", "O"}:
            # strings -> indices 0/1
            classes, inv = np.unique(y, return_inverse=True)
            y_train = inv.astype(np.float32).reshape(-1)
        else:
            y_train = y.astype(np.float32).reshape(-1)
        loss_fn  = tf.keras.losses.BinaryCrossentropy(from_logits=from_logits)
        metrics  = [tf.keras.metrics.BinaryAccuracy(name="binary_accuracy")]
    else:
        # Multiclass sparse
        if y.ndim == 2 and y.shape[1] > 1:
            # one-hot -> class ids
            y_train = y.argmax(axis=1).astype(np.int32).reshape(-1)
        elif y.dtype.kind in {"U", "S", "O"}:
            # strings -> class ids
            classes, inv = np.unique(y, return_inverse=True)
            y_train = inv.astype(np.int32).reshape(-1)
        elif y.dtype.kind == "f":
            # floats -> round to nearest class id
            y_train = np.rint(y).astype(np.int32).reshape(-1)
        else:
            y_train = y.astype(np.int32).reshape(-1)
        loss_fn  = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=from_logits)
        metrics  = [tf.keras.metrics.SparseCategoricalAccuracy(name="sparse_categorical_accuracy")]

    # Compile & train
    model.compile(optimizer=OPTIMIZER, loss=loss_fn, metrics=metrics)
    print(f"→ {user_dir.name}: X{X.shape}, y{y_train.shape}, outputs={num_outputs}, "
          f"from_logits={from_logits}, loss={loss_fn.name}, metric={[m.name for m in metrics]}")

    start = time.time()
    try:
        hist = model.fit(X, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)
    except Exception as e:
        if "same rank" in str(e) or "ranks" in str(e):
            print(f"   ⚠️  {user_dir.name}: metric rank issue -> retrying with no metrics")
            model.compile(optimizer=OPTIMIZER, loss=loss_fn, metrics=[])
            hist = model.fit(X, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)
        else:
            raise
    elapsed = time.time() - start

    # Save weights + params (unchanged)
    out_dir.mkdir(parents=True, exist_ok=True)
    weights_path = out_dir / f"{user_dir.name}.weights.h5"
    model.save_weights(str(weights_path))
    final_metrics = {f"final_{k}": float(v[-1]) for k, v in getattr(hist, "history", {}).items()}
    params = {
        "user": user_dir.name,
        "model_file": model_path.name,
        "samples": int(X.shape[0]),
        "features": int(X.shape[-1]),
        "is_lstm": is_lstm,
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "optimizer": OPTIMIZER,
        "loss": loss_fn.name,
        "metrics": [m.name for m in metrics],
        "train_seconds": round(elapsed, 3),
        **final_metrics
    }
    with open(out_dir / "params.json", "w") as f:
        json.dump(params, f, indent=2)

    return {"weights": str(weights_path), "params": params}



def main():
    session = gh_session()

    # 1) Download exact MC models
    print("🔎 Fetching exact multiclass global models from GitHub…")
    fnn_entry  = get_github_file_entry(session, OWNER, REPO, FNN_MODEL_PATH, REF)
    lstm_entry = get_github_file_entry(session, OWNER, REPO, LSTM_MODEL_PATH, REF)

    cache_dir = pathlib.Path("./_downloaded_models")
    fnn_local  = download_file(session, fnn_entry,  cache_dir / pathlib.Path(FNN_MODEL_PATH).name)
    lstm_local = download_file(session, lstm_entry, cache_dir / pathlib.Path(LSTM_MODEL_PATH).name)
    print(f"⬇️  FNN MC model:  {fnn_local}")
    print(f"⬇️  LSTM MC model: {lstm_local}")

    # 2) Enumerate user folders
    fnn_users  = load_user_dirs(FNN_USERS_DIR)
    lstm_users = load_user_dirs(LSTM_USERS_DIR)

    if not fnn_users:
        print(f"⚠️  No valid FNN user folders found under: {FNN_USERS_DIR}")
    if not lstm_users:
        print(f"⚠️  No valid LSTM user folders found under: {LSTM_USERS_DIR}")

    fnn_times: List[float] = []
    lstm_times: List[float] = []

    # 3) Train FNN users
    print(f"🏃 Training FNN (MC) users ({len(fnn_users)}) …")
    for udir in fnn_users:
        out_dir = FNN_OUT_ROOT / udir.name
        try:
            result = train_one_user(fnn_local, udir, out_dir, is_lstm=False)
            secs = float(result["params"]["train_seconds"])
            fnn_times.append(secs)
            print(f"✅ {udir.name}: {secs:.3f}s  -> {result['weights']}")
        except Exception as e:
            print(f"❌ {udir.name}: {e}")

    # 4) Train LSTM users
    print(f"🏃 Training LSTM (MC) users ({len(lstm_users)}) …")
    for udir in lstm_users:
        out_dir = LSTM_OUT_ROOT / udir.name
        try:
            result = train_one_user(lstm_local, udir, out_dir, is_lstm=True)
            secs = float(result["params"]["train_seconds"])
            lstm_times.append(secs)
            print(f"✅ {udir.name}: {secs:.3f}s  -> {result['weights']}")
        except Exception as e:
            print(f"❌ {udir.name}: {e}")

    # 5) Summaries
    if fnn_times:
        print("\n⏱️ FNN (MC) training time summary:")
        print(f"   Users trained: {len(fnn_times)}")
        print(f"   Avg: {sum(fnn_times)/len(fnn_times):.3f}s   Min: {min(fnn_times):.3f}s   Max: {max(fnn_times):.3f}s")

    if lstm_times:
        print("\n⏱️ LSTM (MC) training time summary:")
        print(f"   Users trained: {len(lstm_times)}")
        print(f"   Avg: {sum(lstm_times)/len(lstm_times):.3f}s   Min: {min(lstm_times):.3f}s   Max: {max(lstm_times):.3f}s")

    print("\n🎉 Done.")


if __name__ == "__main__":
    main()


🔎 Fetching exact multiclass global models from GitHub…
⬇️  FNN MC model:  _downloaded_models/global_FNN_MC_model.keras
⬇️  LSTM MC model: _downloaded_models/global_lstm_MC_model.keras
🏃 Training FNN (MC) users (128) …
→ user_001: X(46, 52), y(46,), outputs=4, from_logits=False, loss=sparse_categorical_crossentropy, metric=['sparse_categorical_accuracy']
✅ user_001: 0.427s  -> /Users/sohinikar/FL/M.Tech_Dissertation/Client/client_params/FNN_MC/user_001/user_001.weights.h5
→ user_002: X(46, 52), y(46,), outputs=4, from_logits=False, loss=sparse_categorical_crossentropy, metric=['sparse_categorical_accuracy']
✅ user_002: 0.421s  -> /Users/sohinikar/FL/M.Tech_Dissertation/Client/client_params/FNN_MC/user_002/user_002.weights.h5
→ user_003: X(46, 52), y(46,), outputs=4, from_logits=False, loss=sparse_categorical_crossentropy, metric=['sparse_categorical_accuracy']
✅ user_003: 0.415s  -> /Users/sohinikar/FL/M.Tech_Dissertation/Client/client_params/FNN_MC/user_003/user_003.weights.h5
→ user_0